In [19]:
# import libraries
import numpy as np
from cryptorandom.sample import random_sample, random_allocation, get_prng
from cryptorandom.cryptorandom import SHA256
import time
import itertools
from collections import Counter
import matplotlib.pyplot as plt
from scipy.stats import norm
import random
import math
import warnings
import sys
import os

### Garthwaite Algorithm

In [5]:
def garthwaite(x, y, initial_guess=None, cl=0.95, reps=10**4, seed=None):
    r"""
    Construct random permutation confidence intervals for the two-sample problem using the algorithm in Garthwaite (1996)

    Parameters
    ----------
    x : array-like
    y : array-like
    initial_guess : array-like
        Search starting points for the lower and upper confidence interval endpoints
    cl : int
        Confidence level
    reps : int
        Number of steps
    seed : RandomState instance or {None, int, RandomState instance}
        If None, the pseudorandom number generator is the RandomState
        instance used by `np.random`;
        If int, seed is the seed used by the random number generator;
        If RandomState instance, seed is the pseudorandom number generator

    Returns
    -------
    tuple
        Returns confidence interval lower bound, confidence interval upper bound, 
        array of the lower bound estimates at each step of the search, array of the upper bound estimates at each step of the search
    """
    # compute alpha from confidence level
    alpha = (1-cl)/2
    full_sample = np.array([*x, *y])
    n = len(full_sample)
    nx = len(x)
    my_prng = get_prng(seed)
    
    # set k
    k = 2 / ((norm.ppf(1-alpha) / np.sqrt(2*math.pi)) * math.exp(-norm.ppf(1-alpha)**2 / 2))

    # set initial CI limits
    if initial_guess == None:
        upper_lim = max(abs(max(x) - min(y)), abs(max(y) - min(x)))
        lower_lim = -upper_lim
    else:
        upper_lim = initial_guess[1]
        lower_lim = initial_guess[0]
        
    # calculate observed difference in means
    obs = np.mean(x) - np.mean(y)
    
    # set c
    c_upper = k*(upper_lim - obs)
    c_lower = k*(obs - lower_lim)
    
    # if c is negative, raise error
    if c_upper < 0 or c_lower < 0:
        raise ValueError("Constant c cannot be negative. Provide different initial guess.")

    # initilize U and L
    U = [0]*reps
    L = [0]*reps
    
    # iterate through steps of Garthwaite's algorithm
    ## Upper limit ##
    for i in range(reps):
        # Potential outcomes for all units under treatment
        pot_outx = np.concatenate([x, y + upper_lim])
        # Potential outcomes for all units under control
        pot_outy = np.concatenate([x - upper_lim, y])
        pot_out_all = np.column_stack([pot_outx, pot_outy])
        rr = list(range(pot_out_all.shape[0]))
        # Just do one permutation 
        my_prng.shuffle(rr)
        pp = np.take(pot_out_all, rr, axis=0)
        # get difference in means for that permutation
        perm_mean_diff = np.mean(pp[:nx, 0]) - np.mean(pp[nx:, 1])
        # update U and L
        if perm_mean_diff > obs:
            U[i] = upper_lim - c_upper*alpha/(i+1)
            upper_lim = U[i]
        else:
            U[i] = upper_lim + c_upper*(1-alpha)/(i+1)
            upper_lim = U[i]
        # set c
        c_upper = k*(upper_lim - obs)
        
    ## Lower limit ##
    for i in range(reps):
        # Potential outcomes for all units under treatment
        pot_outx = np.concatenate([x, y + lower_lim])
        # Potential outcomes for all units under control
        pot_outy = np.concatenate([x - lower_lim, y])
        pot_out_all = np.column_stack([pot_outx, pot_outy])
        rr = list(range(pot_out_all.shape[0]))
        # Just do one permutation 
        my_prng.shuffle(rr)
        pp = np.take(pot_out_all, rr, axis=0)
        # get difference in means for that permutation
        perm_mean_diff = np.mean(pp[:nx, 0]) - np.mean(pp[nx:, 1])
        # update U and L
        if perm_mean_diff < obs:
            L[i] = lower_lim + c_lower*alpha/(i+1)
            lower_lim = L[i]
        else:
            L[i] = lower_lim - c_lower*(1-alpha)/(i+1) 
            lower_lim = L[i]
        # set c
        c_lower = k*(obs-lower_lim)
        
    return lower_lim, upper_lim, L, U

In [6]:
# Garthwaite example
x = np.array([16.4, 29.4, 37.1, 23.0, 24.1, 24.5, 16.4, 29.1, 
              36.7, 28.7, 30.2, 21.8, 37.1, 20.3, 28.3])
y = np.array([22.2, 34.8, 42.1, 32.9, 26.4, 30.6, 32.9, 37.5, 18.4, 
              27.5, 45.5, 34.0, 45.5, 24.5, 28.7])

start_time = time.process_time()
lower_lim, upper_lim, L, U = garthwaite(y, x, initial_guess = [0, 10], cl=0.95, reps=6000, seed=13)
end_time = time.process_time()

print(f"CPU time used: {end_time - start_time} seconds")

print(lower_lim, upper_lim)

CPU time used: 0.5109299999999992 seconds
-0.2883777538941723 10.992864741115884


In [13]:
# timing tests
time_array = [0]*100
for i in range(len(time_array)):
    start_time = time.process_time()
    lower_lim, upper_lim, L, U = garthwaite(y, x, initial_guess = [0, 10], cl=0.95, reps=6000, seed=13)
    end_time = time.process_time()
    time_array[i] = end_time - start_time

In [14]:
np.mean(time_array)

np.float64(0.46571911000000143)

### New method

In [15]:
def calc_one_sample_perm_stat(x, reps=10**5, complete_enum=False, seed=None):
    r"""
    Calculate mean of observed data, mean of reps number of permuted data and corresponding number of negative values in the permutations

    Parameters
    ----------
    x : array-like
    reps : int
        Number of random permutations
    complete_enum : bool
        Whether to enumerate all possible permutations. If True, reps is ignored. Default, False.
    seed : RandomState instance or {None, int, RandomState instance}
        If None, the pseudorandom number generator is the RandomState
        instance used by `np.random`;
        If int, seed is the seed used by the random number generator;
        If RandomState instance, seed is the pseudorandom number generator

    Returns
    -------
    tuple
        Returns mean of observed data x, array of means of reps number of permuted data, 
        array of corresponding number of negative values in each permutation
    """
    # get number of observations
    n = len(x)
    # get observed mean
    obs_mean = np.mean(x)
    # check if complete enumeration
    if complete_enum:
        # get complete set of permutations
        perms = [list(item) for item in itertools.product([-1, 1], repeat=n)]
        # throw warning if complete enumeration will result in more than a million permutations
        if perms > 10**7:
            warnings.warn("Warning: complete enumertation involves more than 1 million permutations")
        # initalize array of permutation means and count of elements that change signs
        perm_mean = [0] * len(perms)
        perm_neg = [0] * len(perms)
        i = 0
        # calculate sample mean and number of -1 for each permutation
        for perm in perms:
            perm_mean[i] = np.mean(x * perm)
            perm_neg[i] = np.sum(np.array(perm) == -1) 
            i = i+1
    else:
        # initalize array of permutation means and count of elements that change signs
        perm_mean = [0] * reps
        perm_neg = [0] * reps
        # set seed
        my_seed = get_prng(seed) 
        # calculate sample mean and number of -1 for each permutation
        for i in range(reps):
            perm = random_sample([1, -1], n, replace=True, prng=my_seed)
            #perm = np.random.choice([1, -1], n, replace=True) # default PRNG
            perm_mean[i] = np.mean(x * perm)
            perm_neg[i] = np.sum(perm == -1)
    return obs_mean, perm_mean, perm_neg

def calc_two_sample_perm_stat(x, y, reps=10**5, seed=None, complete_enum = False):
    r"""
    Calculate difference in means of observed data, 
    difference in means of reps number of permuted datasets and 
    corresponding number of values that switched from 'treatment' to 'control' in the permutations

    Parameters
    ----------
    x : array-like
    y : array-like
    reps : int
        Number of random permutations
    complete_enum : bool
        Whether to enumerate all possible permutations. If True, reps is ignored. Default, False.
    seed : RandomState instance or {None, int, RandomState instance}
        If None, the pseudorandom number generator is the RandomState
        instance used by `np.random`;
        If int, seed is the seed used by the random number generator;
        If RandomState instance, seed is the pseudorandom number generator

    Returns
    -------
    tuple
        Returns difference in means of observed data, 
        difference in means of reps number of permuted datasets and 
        corresponding number of values that switched from 'treatment' to 'control' in the permutations
    """
    # concat x and y to get full sample
    full_sample = np.array([*x, *y])
    # get sample sizes
    nx = len(x)
    ny = len(y)
    n = len(full_sample)
    # get observed difference
    obs_diff = np.mean(x) - np.mean(y)
    # check if complete enumeration
    if complete_enum:
        # array of indices of the full sample
        z = np.arange(n)
        # array of all combinations of indices for x
        poss_perm_indices_x = list(itertools.combinations(z, len(x)))
        # array of corresponding indices for y
        poss_perm_indices_y = [0]*len(poss_perm_indices_x)
        # initialize test statistic and n_tc arrays
        perm_mean_diff = [0]*len(poss_perm_indices_x)
        perm_n_switch = [0]*len(poss_perm_indices_x)
        # calculate difference in means and number of units that switch from treatment to control for each permutation
        for i in range(len(poss_perm_indices_x)):
            diff = Counter(z) - Counter(poss_perm_indices_x[i])
            poss_perm_indices_y[i] = list(diff.elements())
            perm_mean_diff[i] = np.mean(full_sample[np.array(poss_perm_indices_x[i])]) - np.mean(full_sample[np.array(poss_perm_indices_y[i])])
            perm_n_switch[i] = len(list(set(poss_perm_indices_x[i])-set(np.arange(nx))))
    else:
        # get array with difference in means for each permutation and number of control and treated units that switch
        perm_mean_diff = [0] * reps
        perm_n_switch = [0] * reps
        my_seed = get_prng(seed) 
        # calculate difference in means and number of units that switch from treatment to control for each permutation
        for i in range(reps):
            perm_indices_x = random_sample(n, nx, prng=my_seed)
            #perm_indices_x = np.random.choice(n, nx, replace=False) # default PRNG
            perm_indices_y = list(set(np.arange(n))-set(perm_indices_x))            
            perm_mean_diff[i] = np.mean(full_sample[[perm_indices_x]]) - np.mean(full_sample[[perm_indices_y]])
            perm_n_switch[i] = len(list(set(perm_indices_x)-set(np.arange(nx))))
            
    return obs_diff, perm_mean_diff, perm_n_switch

In [16]:
def bisection_method_ci(x, y = [], cl = 0.95, reps = 10**5, complete_enum=False, 
                        rtol=1e-05, atol=1e-08, max_iter=10**5, seed=None):
    r"""
    Construct random permutation confidence intervals for the one- and two-sample problem using method described in Glazer and Stark

    Parameters
    ----------
    x : array-like
    y : array-like
    cl : int
        Confidence level
    reps : int
        Number of random permutations
    complete_enum : bool
        Whether to enumerate all possible permutations. If True, reps is ignored. Default, False.
    rtol : int
    atol : int
    max_iter : int
        Maximum number of bisection method iterations
    seed : RandomState instance or {None, int, RandomState instance}
        If None, the pseudorandom number generator is the RandomState
        instance used by `np.random`;
        If int, seed is the seed used by the random number generator;
        If RandomState instance, seed is the pseudorandom number generator

    Returns
    -------
    tuple
        Returns the lower and upper confidence bounds, and whether the bounds reach the desired tolerance
    """
    # set alpha
    alpha = 1 - cl
    # get length of sample(s)
    nx = len(x)
    ny = len(y)

    # check if one or two sample and get permutation values
    if len(y) == 0:
        if 2**(-nx) <= alpha:
            perm_values = calc_one_sample_perm_stat(x=x, reps=reps, complete_enum=complete_enum, 
                                                    seed=seed)
            upper_lim = max(x)
            med_lim = np.median(x)
            lower_lim = min(x)
        else:
            raise ValueError("Sample size too small")
    else: 
        # get array with difference in means for each permutation and number of control and treated units that switch
        perm_values = calc_two_sample_perm_stat(x=x, y=y, reps=reps, seed=seed, complete_enum=complete_enum)
        # set upper and lower limits for CI
        upper_lim = max(abs(max(x) - min(y)), abs(max(y) - min(x)))
        lower_lim = -upper_lim
        
    # get observed and necessary values for each permutation to calculate test statistic for each shift
    obs = perm_values[0]
    perm_stats = perm_values[1]
    perm_n_switch = perm_values[2]
    
    ## Get CI upper bound ##
    # initialize upper and lower bounds
    if ny == 0:
        u_lower_bound = lower_lim # add test that alpha < 1/2, FIX ME
    else:
        u_lower_bound = lower_lim
    u_upper_bound = upper_lim
    # calculate midpoint of interval
    midpoint = (u_lower_bound + u_upper_bound) / 2
    # initialize count
    count = 0
    # modified bisection method
    while (np.allclose(u_upper_bound, u_lower_bound, rtol=rtol, atol=atol) == False) and (count < max_iter):
        # calculate test statistic for theta = midpoint
        if len(y) == 0:
            shift_perm_stats = np.asarray(perm_stats) + (midpoint * (2*np.asarray(perm_n_switch))) / nx 
        else: 
            shift_perm_stats = np.asarray(perm_stats) + (midpoint * ((1/nx) + (1/ny)))*np.asarray(perm_n_switch)

        # calculate p-value
        if complete_enum:
            upper_p_value = 2*min(np.mean(shift_perm_stats <= obs), np.mean(shift_perm_stats >= obs))
        else:
            upper_p_value = 2*min((np.sum(obs >= shift_perm_stats) + 1) / (reps + 1), (np.sum(obs <= shift_perm_stats) + 1) / (reps + 1))
            
        # update bounds
        if upper_p_value <= alpha:
            u_upper_bound = midpoint
            midpoint = (u_upper_bound + u_lower_bound) / 2
        elif upper_p_value > alpha: 
            u_lower_bound = midpoint
            midpoint = (u_lower_bound + u_upper_bound) / 2

        # up iteration count
        count += 1
        
    # boolean for upper bound success if reaches tolerance
    u_success = np.allclose(u_upper_bound, u_lower_bound, rtol=rtol, atol=atol)
    
    # print number of iterations run to calculate upper bound
    print("Number of iterations for upper bound ", count)
            
    ## Get CI lower bound ##
    # initialize upper and lower bounds
    l_lower_bound = lower_lim
    l_upper_bound = upper_lim 
    # calculate midpoint
    midpoint = (l_lower_bound + l_upper_bound) / 2
    # initialize count
    count = 0
    # modified bisection method
    while (np.allclose(l_upper_bound, l_lower_bound, rtol=rtol, atol=atol) == False) and count < max_iter:
        # calculate test statistic for theta = midpoint
        if len(y) == 0:
            shift_perm_stats = np.asarray(perm_stats) + (midpoint * (2*np.asarray(perm_n_switch))) / nx 
        else: 
            shift_perm_stats = np.asarray(perm_stats) + (midpoint * ((1/nx) + (1/ny)))*np.asarray(perm_n_switch)

        # calculate p-value
        if complete_enum:
            lower_p_value = 2*min(np.mean(shift_perm_stats <= obs), np.mean(shift_perm_stats >= obs))
        else:
            lower_p_value = 2*min((np.sum(obs >= shift_perm_stats) + 1) / (reps + 1), (np.sum(obs <= shift_perm_stats) + 1) / (reps + 1))
        # update bounds
        if lower_p_value < alpha:
            l_lower_bound = midpoint
            midpoint = (l_upper_bound + l_lower_bound) / 2
        elif lower_p_value >= alpha: 
            l_upper_bound = midpoint
            midpoint = (l_lower_bound + l_upper_bound) / 2

        # up iteration count
        count += 1
    
    # boolean for lower bound success if reaches tolerance
    l_success = np.allclose(l_upper_bound, l_lower_bound, rtol=rtol, atol=atol)
    
    # print number of iterations run to calculate lower bound
    print("Number of iterations for lower bound ", count)
    
    return l_lower_bound, u_upper_bound, u_success, l_success

In [61]:
# illustrate PRNG time difference - generate random integers between 1 and 1 million
start_time = time.time()
perm = SHA256(seed = 13).randint(1, 10**7, size=10**7)
print("--- %s seconds ---" % (time.time() - start_time))
# default Python
start_time = time.time()
perm = np.random.randint(1, 10**7, 10**7) # default PRNG
print("--- %s seconds ---" % (time.time() - start_time))

--- 15.951695919036865 seconds ---
--- 0.15510177612304688 seconds ---


In [20]:
# Two-sample problem with Garthwaite data
x = np.array([16.4, 29.4, 37.1, 23.0, 24.1, 24.5, 16.4, 29.1, 
              36.7, 28.7, 30.2, 21.8, 37.1, 20.3, 28.3])
y = np.array([22.2, 34.8, 42.1, 32.9, 26.4, 30.6, 32.9, 37.5, 18.4, 
              27.5, 45.5, 34.0, 45.5, 24.5, 28.7])
# timing tests
time_array = [0]*100
# Save current stdout
original_stdout = sys.stdout
# Redirect stdout to null
sys.stdout = open(os.devnull, 'w')
for i in range(len(time_array)):
    start_time = time.process_time()
    result = bisection_method_ci(y, x, reps= 10**4, cl = 0.95, max_iter=40, seed = 13)
    end_time = time.process_time()
    time_array[i] = end_time - start_time
# Restore original stdout
sys.stdout = original_stdout

In [21]:
np.mean(time_array)

np.float64(0.32549246000000065)

In [50]:
# One-sample problem example with Tritchler data, complete enumeration
start_time = time.time()
print(bisection_method_ci(x = np.array([49, -67, 8, 6, 16, 23, 28, 41, 14, 29, 56, 24, 75, 60, -48]), 
                    cl = 0.90, complete_enum=True)) 
print("--- %s seconds ---" % (time.time() - start_time))

Number of iterations for upper bound  19
Number of iterations for lower bound  22
(3.7499775886535645, 38.14307403564453, True, True)
--- 0.6279029846191406 seconds ---


In [77]:
# One-sample problem example with Tritchler data, 10**4 random permutations
start_time = time.time()
print(bisection_method_ci(x = np.array([49, -67, 8, 6, 16, 23, 28, 41, 14, 29, 56, 24, 75, 60, -48]), 
                    cl = 0.90, reps = 10**4, complete_enum=False, seed=13)) 
print("--- %s seconds ---" % (time.time() - start_time))

print(bisection_method_ci(x = np.array([49, -67, 8, 6, 16, 23, 28, 41, 14, 29, 56, 24, 75, 60, -48]), 
                    cl = 0.95, reps = 10**4, complete_enum=False, seed=13)) 

print(bisection_method_ci(x = np.array([49, -67, 8, 6, 16, 23, 28, 41, 14, 29, 56, 24, 75, 60, -48]), 
                    cl = 0.99, reps = 10**4, complete_enum=False, seed=13)) 

Number of iterations for upper bound  19
Number of iterations for lower bound  22
(3.8571300506591797, 38.250057220458984, True, True)
--- 0.49468493461608887 seconds ---
Number of iterations for upper bound  19
Number of iterations for lower bound  27
(-0.16666707396507263, 41.00020217895508, True, True)
Number of iterations for upper bound  19
Number of iterations for lower bound  21
(-8.800064086914062, 47.20008087158203, True, True)


In [22]:
# One-sample problem example with Tritchler data, 10**4 random permutations
# timing tests
time_array = [0]*100
# Save current stdout
original_stdout = sys.stdout
# Redirect stdout to null
sys.stdout = open(os.devnull, 'w')
for i in range(len(time_array)):
    start_time = time.process_time()
    result = bisection_method_ci(x = [49, -67, 8, 6, 16, 23, 28, 41, 14, 29, 56, 24, 75, 60, -48],
                                 reps= 10**4, cl = 0.95, max_iter=40, seed = 13)
    end_time = time.process_time()
    time_array[i] = end_time - start_time
# Restore original stdout
sys.stdout = original_stdout

In [23]:
np.mean(time_array)

np.float64(0.23168391000000013)

In [36]:
# One-sample problem example with Tritchler data, 10**4 random permutations
start_time = time.process_time()
print(bisection_method_ci(x = np.array([49, -67, 8, 6, 16, 23, 28, 41, 14, 29, 56, 24, 75, 60, -48, 
                                       49, -67, 8, 6, 6, 23, 28, 41,14,29]), 
                    cl = 0.95, reps = 10**4, seed=13))
end_time = time.process_time()

print(f"CPU time used: {end_time - start_time} seconds")

Number of iterations for upper bound  19
Number of iterations for lower bound  22
(3.5999979972839355, 32.46158981323242, True, True)
CPU time used: 0.31812699999989036 seconds


In [24]:
# One-sample problem example with Tritchler additional data, 10**4 random permutations
# timing tests
time_array = [0]*100
# Save current stdout
original_stdout = sys.stdout
# Redirect stdout to null
sys.stdout = open(os.devnull, 'w')
for i in range(len(time_array)):
    start_time = time.process_time()
    result = bisection_method_ci(x = [49, -67, 8, 6, 16, 23, 28, 41, 14, 29, 56, 24, 75, 60, -48, 
                                       49, -67, 8, 6, 6, 23, 28, 41,14,29],
                                 reps= 10**4, cl = 0.95, max_iter=40, seed = 13)
    end_time = time.process_time()
    time_array[i] = end_time - start_time
# Restore original stdout
sys.stdout = original_stdout

In [25]:
np.mean(time_array)

np.float64(0.29361397000000095)

In [79]:
# Two-sample problem example with Tritchler data, 10**4 random permutations
x = np.array([35.3, 35.9, 37.2, 33.0, 31.9, 33.7, 36.0, 35.0, 
              33.3, 33.6, 37.9, 35.6, 29.0, 33.7, 35.7])
y = np.array([32.5, 34.0, 34.4, 31.8, 35.0, 34.6, 33.5, 33.6, 31.5, 33.8, 34.6])

start_time = time.process_time()
print(bisection_method_ci(y, x, reps= 10**4, cl = 0.90, max_iter=40, seed = 13))
end_time = time.process_time()
print(f"CPU time used: {end_time - start_time} seconds")

print(bisection_method_ci(y, x, reps= 10**4, cl = 0.95, max_iter=40, seed = 13))
print(bisection_method_ci(y, x, reps= 10**4, cl = 0.99, max_iter=40, seed = 13))

/var/folders/d_/d_l8pffd5g9339qj6mktkq580000gn/T/ipykernel_23265/1152499453.py:120: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  perm_mean_diff[i] = np.mean(full_sample[[perm_indices_x]]) - np.mean(full_sample[[perm_indices_y]])


Number of iterations for upper bound  22
Number of iterations for lower bound  20
(-2.1166748046874995, 0.38000183105468727, True, True)
CPU time used: 0.5828760000000557 seconds
Number of iterations for upper bound  21
Number of iterations for lower bound  20
(-2.3333374023437496, 0.6428588867187499, True, True)
Number of iterations for upper bound  21
Number of iterations for lower bound  19
(-2.8500244140624997, 1.1666687011718748, True, True)


In [26]:
# Two-sample problem example with Tritchler data, 10**4 random permutations
x = np.array([35.3, 35.9, 37.2, 33.0, 31.9, 33.7, 36.0, 35.0, 
              33.3, 33.6, 37.9, 35.6, 29.0, 33.7, 35.7])
y = np.array([32.5, 34.0, 34.4, 31.8, 35.0, 34.6, 33.5, 33.6, 31.5, 33.8, 34.6])
# timing tests
time_array = [0]*100
# Save current stdout
original_stdout = sys.stdout
# Redirect stdout to null
sys.stdout = open(os.devnull, 'w')
for i in range(len(time_array)):
    start_time = time.process_time()
    result = bisection_method_ci(y, x,
                                 reps= 10**4, cl = 0.95, max_iter=40, seed = 13)
    end_time = time.process_time()
    time_array[i] = end_time - start_time
# Restore original stdout
sys.stdout = original_stdout

In [27]:
np.mean(time_array)

np.float64(0.2758584599999992)

In [72]:
# Two-sample problem example with Tritchler data, sample size doubled, 10**4 random permutations
x = np.array([35.3, 35.9, 37.2, 33.0, 31.9, 33.7, 36.0, 35.0, 
              33.3, 33.6, 37.9, 35.6, 29.0, 33.7, 35.7,
             35.3, 35.9, 37.2, 33.0, 31.9, 33.7, 36.0, 35.0, 
              33.3, 33.6, 37.9, 35.6, 29.0, 33.7, 35.7])
y = np.array([32.5, 34.0, 34.4, 31.8, 35.0, 34.6, 33.5, 33.6, 31.5, 33.8, 34.6, 
             32.5, 34.0, 34.4, 31.8, 35.0, 34.6, 33.5, 33.6, 31.5, 33.8, 34.6])

start_time = time.process_time()
print(bisection_method_ci(y, x, reps= 10**4, cl = 0.90, max_iter=40, seed = 13))
end_time = time.process_time()
print(f"CPU time used: {end_time - start_time} seconds")

print(bisection_method_ci(y, x, reps= 10**4, cl = 0.95, max_iter=40, seed = 13))
print(bisection_method_ci(y, x, reps= 10**4, cl = 0.99, max_iter=40, seed = 13))

/var/folders/d_/d_l8pffd5g9339qj6mktkq580000gn/T/ipykernel_23265/1152499453.py:120: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  perm_mean_diff[i] = np.mean(full_sample[[perm_indices_x]]) - np.mean(full_sample[[perm_indices_y]])


Number of iterations for upper bound  18
Number of iterations for lower bound  18
(6.399951171874998, -6.399951171874998, True, True)
CPU time used: 0.815827000000013 seconds
Number of iterations for upper bound  24
Number of iterations for lower bound  20
(-1.8833374023437495, 0.1416671752929687, True, True)
Number of iterations for upper bound  22
Number of iterations for lower bound  20
(-2.1666748046874993, 0.4300018310546873, True, True)


In [28]:
# Two-sample problem example with Tritchler additional data, 10**4 random permutations
x = np.array([35.3, 35.9, 37.2, 33.0, 31.9, 33.7, 36.0, 35.0, 
              33.3, 33.6, 37.9, 35.6, 29.0, 33.7, 35.7,
             35.3, 35.9, 37.2, 33.0, 31.9, 33.7, 36.0, 35.0, 
              33.3, 33.6, 37.9, 35.6, 29.0, 33.7, 35.7])
y = np.array([32.5, 34.0, 34.4, 31.8, 35.0, 34.6, 33.5, 33.6, 31.5, 33.8, 34.6, 
             32.5, 34.0, 34.4, 31.8, 35.0, 34.6, 33.5, 33.6, 31.5, 33.8, 34.6])
# timing tests
time_array = [0]*100
# Save current stdout
original_stdout = sys.stdout
# Redirect stdout to null
sys.stdout = open(os.devnull, 'w')
for i in range(len(time_array)):
    start_time = time.process_time()
    result = bisection_method_ci(y, x,
                                 reps= 10**4, cl = 0.95, max_iter=40, seed = 13)
    end_time = time.process_time()
    time_array[i] = end_time - start_time
# Restore original stdout
sys.stdout = original_stdout

In [29]:
np.mean(time_array)

np.float64(0.45321285000000017)

In [181]:
# Two-sample problem example with Tritchler data, complete enumeration, 90% CI
x = np.array([35.3, 35.9, 37.2, 33.0, 31.9, 33.7, 36.0, 35.0, 
              33.3, 33.6, 37.9, 35.6, 29.0, 33.7, 35.7])
y = np.array([32.5, 34.0, 34.4, 31.8, 35.0, 34.6, 33.5, 33.6, 31.5, 33.8, 34.6])

start_time = time.process_time()
print(bisection_method_ci(y, x, complete_enum=True, cl = 0.90, max_iter=40, seed = 13))
end_time = time.process_time()
print(f"CPU time used: {end_time - start_time} seconds")

Number of iterations for upper bound  22
Number of iterations for lower bound  20
(-2.1142944335937504, 0.3857147216796874, True, True)
CPU time used: 312.5978930000001 seconds


In [ ]:
# Two-sample problem example with Tritchler data, complete enumerationm, 95% CI
start_time = time.process_time()
print(bisection_method_ci(y, x, reps= 10**4, complete_enum=True, cl = 0.95, max_iter=40, seed = 13))
end_time = time.process_time()
print(f"CPU time used: {end_time - start_time} seconds")

Number of iterations for upper bound  21
Number of iterations for lower bound  20
(-2.3400024414062486, 0.6499999999999999, True, True)
CPU time used: 315.33381199999985 seconds


In [183]:
# Two-sample problem example with Tritchler data, complete enumeration, 99% CI
start_time = time.process_time()
print(bisection_method_ci(y, x, reps= 10**4, complete_enum=True, cl = 0.99, max_iter=40, seed = 13))
end_time = time.process_time()
print(f"CPU time used: {end_time - start_time} seconds")

Number of iterations for upper bound  21
Number of iterations for lower bound  19
(-2.8143066406249995, 1.1800048828125003, True, True)
CPU time used: 315.92015200000014 seconds


In [37]:
# Run time for large synthetic example
x = np.random.normal(0, 1, 1000)
y = np.random.normal(0, 1, 1000)
start_time = time.process_time()
print(bisection_method_ci(x, y, reps= 10**4, complete_enum=False, cl = 0.95, max_iter=40, seed = 13))
end_time = time.process_time()
print(f"CPU time used: {end_time - start_time} seconds")

/var/folders/d_/d_l8pffd5g9339qj6mktkq580000gn/T/ipykernel_23265/3949393349.py:120: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  perm_mean_diff[i] = np.mean(full_sample[[perm_indices_x]]) - np.mean(full_sample[[perm_indices_y]])


Number of iterations for upper bound  24
Number of iterations for lower bound  25
(-0.07352474082022088, 0.10085510888778107, True, True)
CPU time used: 5.219944000000055 seconds
